<a href="https://colab.research.google.com/github/sasuraibito1125/google_colab/blob/main/%E6%B0%97%E8%B1%A1%E5%BA%81%E3%81%AE%E9%81%8E%E5%8E%BB%E3%81%AE%E6%B0%97%E8%B1%A1%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E3%83%AD%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 気象庁データのロード

下記サイトのデータをロードして`DataFrame`にするためのユーティリティ関数。

[気象庁｜過去の気象データ検索](https://www.data.jma.go.jp/obd/stats/etrn/index.php)

## 関数定義

In [1]:
#@title #### 気象庁データのロード関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名: `download_jmt_data`
#@markdown * 引数：`url_map`, `csv_file='data.csv'`, `each_shaper=lambda place`, `df: df`, `post_shaper=lambda df: df`, `header=4`
#@markdown * 戻り値：`DataFrame`
def download_jmt_data(url_map, csv_file='data.csv',
                      each_shaper=lambda place, df: df,
                      post_shaper=lambda df: df,
                      header=4):
  '''気象庁からHTMLをダウンロードしてCSVファイルを作成する。
  CSVファイルがある場合はダウンロードは行わずにCSVファイルをロードする。

  Parameters
  ----------
  url_map : dict
      都市名とデータサイトURLのリスト。キーが都市名、値が各気象庁データ（検索結果サイト）のURL。
      結果は項目によって複数に分割されている（気温と雲量など）ので、必要な項目が含まれるものを全て列挙する。
  csv_file : str
      出力するCSVファイル名。デフォルトは'data.csv'。
  each_shaper : lambda
      各都市のデータに対するデータ整形処理。デフォルトは何もしない。
  post_shaper : lambda
      最終的なデータ整形処理。デフォルトは何もしない。
  header : int
      ヘッダ行数。デフォルトは4行。データロードされる場合は実際のデータからの値で更新する。

  Returns
  -------
  DataFrame
      最終的に整形されたpandasのDataFrame。
      都市毎のDataFrameを結合した形式。

  '''
  import pandas as pd
  import os
  if not os.path.isfile(csv_file):
    print(f'CSVファイル({csv_file})がないため、気象庁からデータを取得して作成します...\n')
    df = post_shaper(pd.concat([each_shaper(e[0], pd.concat([pd.read_html(url)[0] for url in e[1]], axis=1)) for e in url_map.items()], axis=1))
    header = df.columns.nlevels # ensure_datafileは未対応
    df.to_csv(csv_file)
    print(f'CSVファイル({csv_file})への書き込みが完了しました。続けてCSVファイルを読み込みます...')
  else:
    print(f'CSVファイル({csv_file})が存在するため、ダウンロードをせずにファイルを読み込みます...')
  return pd.read_csv(csv_file, header=list(range(header)), index_col=[0])
  # return pd.read_csv(csv_file, header=[0, 1])


## 利用例

In [4]:
#@title 2022年7月の東京・札幌・那覇の気象データ（平均気温・降水量・平均雲量）のロード { display-mode: "form" }
url_map = {
  # '東京': [ # 日本語版
  'Tokyo': [
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2022&month=07&day=1&view=p1',
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2022&month=07&day=1&view=a4'
  ],
  # '札幌': [ # 日本語版
  'Sapporo': [
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=14&block_no=47412&year=2022&month=07&day=1&view=p1',
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=14&block_no=47412&year=2022&month=07&day=1&view=a4'
  ],
  # '那覇': [ # 日本語版
  'Naha': [
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=91&block_no=47936&year=2022&month=7&day=1&view=p1',
      'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=91&block_no=47936&year=2022&month=7&day=1&view=a4'
  ]
}

def each_shaper(place, df):
  '''各データに対する整形処理

  Paramters
  ---------
  place: target place name
  df: input data frame

  Returns
  -------
  DataFrame
    shaped data
  '''
  import pandas as pd
  df = df.filter(regex='(平均|合計)').filter(regex='(気温|降水量|雲量)').replace('--', 0.0).iloc[:, [1, 0, 2]]
  # df.columns = pd.MultiIndex.from_tuples([(place, we) for we in ['平均気温', '降水量', '平均雲量']]) # 日本語版
  df.columns = pd.MultiIndex.from_tuples([(place, we) for we in ['Temperature', 'Precipitation', 'CloudCover']])
  # df[(place, '降水量')] = df[(place, '降水量')].astype('float64') # 日本語版
  df[(place, 'Precipitation')] = df[(place, 'Precipitation')].astype('float64')
  return df

def post_shaper(df):
  '''結合後のデータに対する整形処理

  Paramters
  ---------
  df: input data frame

  Returns
  -------
  DataFrame
    shaped data
  '''
  import pandas as pd
  import datetime
  df['date'] = pd.Series([datetime.date(2022, 7, d+1) for d in df.index])
  df.set_index('date', inplace=True)
  return df

csv_file = 'weather.csv'
df = download_jmt_data(url_map, csv_file, each_shaper, post_shaper, header=2)
df

CSVファイル(weather.csv)がないため、気象庁からデータを取得して作成します...

CSVファイル(weather.csv)への書き込みが完了しました。続けてCSVファイルを読み込みます...


Tokyo                              Sapporo                \
           Temperature Precipitation CloudCover Temperature Precipitation   
date                                                                        
2022-07-01        30.4           0.0        0.5        20.4           0.0   
2022-07-02        29.5           0.0        5.0        21.5           0.0   
2022-07-03        28.9           0.0        8.3        25.9           0.0   
2022-07-04        26.5           3.5        9.3        24.8           0.0   
2022-07-05        26.2           2.0        9.8        23.6           0.0   
2022-07-06        26.0           0.0        9.5        24.8           0.0   
2022-07-07        25.5           0.0        8.0        25.0           0.0   
2022-07-08        26.0           0.0        7.5        23.2           0.0   
2022-07-09        26.6           0.0        7.5        24.2           0.0   
2022-07-10        27.4           0.0        6.8        24.8           0.5   
2022-07-11        28.1           0.0        8.0        24.3           0.0   
2022-07-12        25.9          32.0       10.0        24.4           0.5   
2022-07-13        23.7          53.0       10.0        23.8           0.0   
2022-07-14        23.8          22.5       10.0        22.9           0.0   
2022-07-15        23.8          61.5       10.0        21.2           0.0   
2022-07-16        23.5           9.0       10.0        20.3           1.5   
2022-07-17        25.4           1.5        7.3        20.3          49.0   
2022-07-18        27.5           0.0        8.0        19.8           0.5   
2022-07-19        27.4           0.0       10.0        23.2           0.0   
2022-07-20        29.7           0.0        8.5        22.9           1.5   
2022-07-21        28.1           0.0        9.8        21.8           0.0   
2022-07-22        28.3           1.5        5.8        22.0           2.0   
2022-07-23        28.7           0.0        5.5        21.2           7.5   
2022-07-24        28.3           0.0        6.3        20.9           0.0   
2022-07-25        29.2           0.0        6.3        21.2           0.0   
2022-07-26        26.6          45.5       10.0        23.2           0.0   
2022-07-27        29.4           0.0        8.5        24.1           0.0   
2022-07-28        28.8           1.0        7.3        23.6           0.0   
2022-07-29        29.1           0.0        6.3        24.3           0.0   
2022-07-30        29.9           0.0        6.8        25.3           0.0   
2022-07-31        30.8           0.0        3.5        26.0           0.0   

                             Naha                           
           CloudCover Temperature Precipitation CloudCover  
date                                                        
2022-07-01        8.8        27.1           7.0        9.5  
2022-07-02        7.5        26.8          26.5        9.5  
2022-07-03        5.0        26.5          33.5        9.0  
2022-07-04        8.0        28.9           0.0        7.3  
2022-07-05        7.0        28.7           0.5        6.5  
2022-07-06        5.0        28.7           2.0        6.5  
2022-07-07        1.8        29.2           0.0        7.0  
2022-07-08        0.5        29.3           0.0        6.8  
2022-07-09        8.8        29.8           1.0        8.8  
2022-07-10        5.3        30.1           0.0        3.5  
2022-07-11        7.5        30.0           0.0        4.5  
2022-07-12       10.0        30.0           0.0        8.0  
2022-07-13       10.0        30.3           0.0        5.0  
2022-07-14        7.3        30.2           0.0        8.0  
2022-07-15       10.0        30.1           0.0        9.8  
2022-07-16       10.0        29.5           8.5        9.8  
2022-07-17       10.0        29.8           0.5       10.0  
2022-07-18       10.0        29.8           2.5        9.3  
2022-07-19        7.8        30.0           1.5        7.8  
2022-07-20        7.3        30.2           0.0        6